<a href="https://colab.research.google.com/github/FlorDGM13/tecnico/blob/main/2_Prueba_tecnica_Etribe.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1.Cargar archivo y explorar estructura

In [ ]:
# 1. Importar librerías necesarias
import pandas as pd
import numpy as np

# 2. Leer todas las hojas del archivo Excel
archivo = "PRUEBA_TECNICA_ETRIBE.xlsx"
excel_file = pd.ExcelFile(archivo)
df_compras = excel_file.parse('COMPRAS')
df_ventas = excel_file.parse('VENTAS')


# 2. VALIDACION DE DATOS

In [ ]:
# Función para validar los DF
def validar_df(df, nombre):
    print(f"\n--- Validación para: {nombre} ---")

    df_clean = df.copy()

    # Validar FECHA_DE_CIERRE
    df_clean['FECHA_DE_CIERRE'] = pd.to_datetime(df_clean['FECHA_DE_CIERRE'], errors='coerce')
    fechas_invalidas = df_clean['FECHA_DE_CIERRE'].isna().sum()
    print(f"Fechas inválidas encontradas: {fechas_invalidas}")

    # Redondear montos
    df_clean['PRECIO'] = df_clean['PRECIO'].round(2)
    df_clean['COSTE_TOTAL'] = df_clean['COSTE_TOTAL'].round(2)

    # Validar cálculo de COSTE_TOTAL
    df_clean['COSTE_TOTAL_CALCULADO'] = (df_clean['CANTIDAD'] * df_clean['PRECIO']).round(2)
    df_clean['COSTE_CORRECTO'] = np.isclose(df_clean['COSTE_TOTAL'], df_clean['COSTE_TOTAL_CALCULADO'])

    # Mostrar errores
    errores = df_clean[~df_clean['COSTE_CORRECTO']]
    print(f"Registros con error en COSTE_TOTAL: {errores.shape[0]}")

    return df_clean, errores

# Ejecutar por separado
df_compras_clean, errores_compras = validar_df(df_compras, "COMPRAS")
df_ventas_clean, errores_ventas = validar_df(df_ventas, "VENTAS")

# Mostrar los errores si los hay
if not errores_compras.empty:
    print("\nErrores en COMPRAS:")
    print(errores_compras.to_string(index=False))

if not errores_ventas.empty:
    print("\nErrores en VENTAS:")
    print(errores_ventas.to_string(index=False))



--- Validación para: COMPRAS ---
Fechas inválidas encontradas: 0
Registros con error en COSTE_TOTAL: 0

--- Validación para: VENTAS ---
Fechas inválidas encontradas: 0
Registros con error en COSTE_TOTAL: 0


In [ ]:
#Calcular el precio promedio por año


df_ventas_clean['ANO'] = pd.to_numeric(df_ventas_clean['ANO'], errors='coerce')
df_compras_clean['ANO'] = pd.to_numeric(df_compras_clean['ANO'], errors='coerce')

# Calcular precio promedio
precio_promedio_ventas = df_ventas_clean.groupby('ANO')['PRECIO'].mean().round(2)
precio_promedio_compras = df_compras_clean.groupby('ANO')['PRECIO'].mean().round(2)

print("\n Precio promedio por AÑO de entrega - VENTAS:")
print(precio_promedio_ventas)

print("\n Precio promedio por AÑO de entrega - COMPRAS:")
print(precio_promedio_compras)



#Calcular el total de piezas vendidas y compradas por año

piezas_vendidas_por_ano = df_ventas_clean.groupby('ANO')['CANTIDAD'].sum()
piezas_compradas_por_ano = df_compras_clean.groupby('ANO')['CANTIDAD'].sum()

print("\n Total de piezas VENDIDAS por AÑO de entrega:")
print(piezas_vendidas_por_ano)

print("\n Total de piezas COMPRADAS por AÑO de entrega:")
print(piezas_compradas_por_ano)



 Precio promedio por AÑO de entrega - VENTAS:
ANO
2025    11.47
2026    10.69
2027     9.42
2028    11.66
2029    11.89
2030    10.07
Name: PRECIO, dtype: float64

 Precio promedio por AÑO de entrega - COMPRAS:
ANO
2025    20.63
2026    17.47
2027    23.22
2028    15.94
2029    31.88
2030    44.87
Name: PRECIO, dtype: float64

 Total de piezas VENDIDAS por AÑO de entrega:
ANO
2025    1950
2026     140
2027      60
2028     140
2029     100
2030     100
Name: CANTIDAD, dtype: int64

 Total de piezas COMPRADAS por AÑO de entrega:
ANO
2025    320
2026     18
2027      8
2028     46
2029     19
2030     22
Name: CANTIDAD, dtype: int64


# 3. EXPORTACION